# Introduction to process discovery (Directly Follow Graph)

By: Jakub Kot, Piotr Kubala, Rafał Łukosz, Piotr Karaś, Tomasz Kawiak

Today's lab builds upon the codes developed in the previous lab.

## 1. [10 points] Automatic CSV Column Matching

Using ipywidgets, extend your existing interface from the previous lab by adding the ability to upload a CSV file with event data. Your solution should attempt to automatically detect and match the columns corresponding to case ID, activity, and timestamp. If automatic matching fails, the user should be prompted to manually select the appropriate columns (e.g., using dropdown lists).

When implementing the matching logic, consider:

- Partial matches in column names,

- The data types in each column,

- Rules or heuristics that should apply for each element (case ID, activity, timestamp).

Define and document:

- The key features used for column detection,

- The method of assessing the best match, especially in logs without headers.

After automatic or manual matching, the user should be able to confirm or adjust the selected columns such as case_id, activity_key, and timestamp_key from the available subset of columns in the log.


## 2. [5 points] Testing

Test your solution on various logs and include in your report a summary table describing the matching results.
For each log, provide:

- The name of the log,

- The total number of columns,

- Which fields were detected,
all fields were detected correctly

- Whether user intervention was required,
no user intervention, however we might have overfitted our assumptions a lil bit :3


- Comments on any detection issues.
sepsis.csv has no start timestamp, so we decided to use the end timestamp as both start and end timestamps.

we have some wild assumptions like the first column is case id (if it's not, we ask the user to select it manually), but we provide our own propositions

same thingy with activity column - we assume it's the second column (if it's not, ask for confirmation)

there's that one csv with no headers so we had to detect if headers are present in the csv, and if no then create generic column names for it


Use the logs available in the “Sample logs” folder on MS Teams.
Test your solution on both the previously used logs (logExample, purchasingExample, repairExample, sepsis) with and without headers, and on the newly added logs with non-standard or missing labels:

new_example_changed_labels.csv, new_teleclaims_changed_labels.csv, new_reviewing_no_column_names.csv.


## 3. [15 points] Improving the Filtering Method

Based on the observations from previous labs — particularly regarding threshold selection and issues with model filtering — refine your filtering method to improve model quality.
Test the new functionality and evaluate whether the filtered models show better results.

In your report, describe at least three improvements, each with:

- The filtering thresholds used,

- An example diagram (or a part of it) before and after the improvement,

- A brief explanation of the change and its effect,

- A code excerpt showing the implementation.

If your previous reports missed certain observations or requirements, consider addressing some of the following ones:

- Ensuring no task becomes detached from the model (each task should remain connected).

- Ensuring all tasks are part of a path from process start to process end.

- Minimizing the number of flows.

- Maximizing the total flow frequency.


Note:
This week, I will collect, summarize, and publish the observations and problems reported in your previous reports.
During the next class (18.11), there will be no new report — instead, we will test your solutions using various logs and thresholds. Therefore, at least one person from each group must attend the next class in person (in classroom C2 316, on-site attendance).
Remember to display after each filtering the process model, the number of edges (flows), and the total sum of labels in the remaining flows. 
In this challenge, we will compare the results, and the winning groups demonstrating the most robust tools will receive bonus points for the labs!

In [125]:
import pandas as pd
from itertools import chain
from itertools import pairwise
from collections import Counter
import pygraphviz as pgv
from IPython.display import Image, display
import ipywidgets as widgets
from collections import OrderedDict
import csv
import os

In [126]:
csvs = ["logs/logExample.csv",
        "logs/new_example_changed_labels.csv",
        "logs/new_reviewing_no_column_names.csv",
        "logs/new_teleclaims_changed_labels.csv", 
        "logs/purchasingExample.csv",
        "logs/repairExample.csv",
        "logs/sepsis.csv"]

In [127]:
def check_header(file_path, bytes = 8192):
    sniffer = csv.Sniffer()
    data = open(file_path, "r").read(bytes)
    has_header = sniffer.has_header(data)
    return has_header

In [128]:
def retrieve_column_names(csv_file: str) -> tuple[pd.DataFrame, str, str, str, str]:
    """
    Retrieve column names for start date, end date, case ID, and activity from a CSV log file.
    How does it work:
    1. Load the CSV file into a DataFrame.
    2. 
    3. Done

    Returns:
    Dataframe, Start date, End date, Case ID, Activity column names
    If a column name was not found, None is returned in its place.
    """


    # check if the first row of csv properly contains column names
    has_header = check_header(csv_file)

    df = pd.read_csv(csv_file, header=0 if has_header else None)  # Load your log file here

    if not has_header:
        # assign generic column names
        df.columns = [f"col_{i}" for i in range(len(df.columns))]

    # convert Start Date to datetime
    for column in df.columns:
        try:
            # check if column can be converted to int (if yes, skip)
            try:
                _ = df[column].dropna().astype(int)
                continue
            except:
                df[column] = pd.to_datetime(df[column], format="mixed")
        except:
            pass

    datetime_columns = df.select_dtypes(include=['datetime64[ns]']).columns

    if len(datetime_columns) == 2:
        # do first column minus second column, if it's negative then first column is start date
        if ((df[datetime_columns[0]] - df[datetime_columns[1]]).dt.total_seconds() > 0).sum() > ((df[datetime_columns[0]] - df[datetime_columns[1]]).dt.total_seconds() < 0).sum():

            start_date_column = datetime_columns[1]
            end_date_column = datetime_columns[0]
        else:
            start_date_column = datetime_columns[0]
            end_date_column = datetime_columns[1]

    else:
        start_date_keywords = ['start', 'begin', 'open', 'created', 'initiated']
        end_date_keywords = ['end', 'close', 'completed', 'finished', 'complete', 'resolved']
        datetime_columns = df.select_dtypes(include=['datetime64[ns]']).columns
        start_date_column = None
        for col in datetime_columns:
            if any(keyword in col.lower() for keyword in start_date_keywords):
                start_date_column = col
                break

        end_date_column = None
        for col in datetime_columns:
            if any(keyword in col.lower() for keyword in end_date_keywords):
                end_date_column = col
                break

        if len(datetime_columns) == 1:
            if start_date_column is None and end_date_column is not None:
                start_date_column = end_date_column
            elif end_date_column is None and start_date_column is not None:
                end_date_column = start_date_column

        if start_date_column is None and end_date_column is None:
            # MANUAL!!
            ...

    case_id_column = None

    # check amount of rows
    if len(df) < 200:
        # MANUAL
        ...
    elif any(any(keyword in col.lower() for keyword in ['case id', 'case_id']) for col in df.columns if col not in datetime_columns):
        # find the column that contains the keyword
        for col in df.columns:
            if any(keyword in col.lower() for keyword in ['case id', 'case_id']):
                case_id_column = col
                break
    else:
        case_id_column = sorted(
            [(i, col, df[col].nunique() if col not in datetime_columns else 0) for i, col in enumerate([col for col in df.columns if col not in datetime_columns])],
            key=lambda x: (x[2], -x[0])
        )[-1][1]
        if case_id_column != 0:
            # MANUAL
            ...

    potential_columns = set(datetime_columns)
    potential_columns.add(case_id_column)

    # for each column count (except case id and datetime) in how many unique case IDs it appears
    activity_columns = [col for col in df.columns if col not in potential_columns]
    # compute scoring per column (sum of distinct values per case as before) and produce ranking
    if case_id_column is not None:
        activity_scores = {col: df.groupby(case_id_column)[col].nunique().sum() for col in activity_columns}
    else:
        activity_scores = {col: df[col].nunique() for col in activity_columns}
    activity_rank = pd.Series(activity_scores).sort_values(ascending=False)

    # keep the top candidate for backward compatibility
    activity_column = activity_rank.index[0]

    return df, start_date_column, end_date_column, case_id_column, activity_column

In [129]:
for csv_file in csvs:
    print(f"Processing file: {csv_file}")
    df, start_date_col, end_date_col, case_id_col, activity_col = retrieve_column_names(csv_file)
    print(f"Total columns: {len(df.columns)}")
    print(f"  Start Date: {start_date_col}")
    print(f"  End Date: {end_date_col}")
    print(f"  Case ID: {case_id_col}")
    print(f"  Activity: {activity_col}")
    print()

Processing file: logs/logExample.csv
Total columns: 9
  Start Date: Start Date
  End Date: End Date
  Case ID: Case ID
  Activity: Activity

Processing file: logs/new_example_changed_labels.csv
Total columns: 9
  Start Date: When Start
  End Date: When End
  Case ID: Instance
  Activity: Task

Processing file: logs/new_reviewing_no_column_names.csv
Total columns: 15
  Start Date: col_3
  End Date: col_4
  Case ID: col_0
  Activity: col_1

Processing file: logs/new_teleclaims_changed_labels.csv
Total columns: 13
  Start Date: from
  End Date: to
  Case ID: id
  Activity: action

Processing file: logs/purchasingExample.csv
Total columns: 6
  Start Date: Start Timestamp
  End Date: Complete Timestamp
  Case ID: Case ID
  Activity: Activity

Processing file: logs/repairExample.csv
Total columns: 17
  Start Date: Start Timestamp
  End Date: Complete Timestamp
  Case ID: Case ID
  Activity: Activity

Processing file: logs/sepsis.csv
Total columns: 34
  Start Date: Complete Timestamp
  End Da

In [130]:
csv_name = "logs/new_teleclaims_changed_labels.csv"

In [131]:
def format_label_value(val, mode):
  if mode == 'coverage':
    return f"{val:.1f}%"
  else:
    try:
      return str(int(val))
    except Exception:
      return str(val)

In [132]:
example_name = "logs/logExample.csv"

In [133]:
df, start_date_col, end_date_col, case_id_col, activity_col = retrieve_column_names(example_name)
#TODO: do this on a new dataframe instead of this one
df['start'] = pd.to_datetime(df[start_date_col])
df['Case ID'] = df[case_id_col]
df['Activity'] = df[activity_col]
dfs = df[['Case ID', 'Activity', 'start']]

In [134]:
# totals and per-case counts
num_cases = dfs['Case ID'].nunique()
# absolute occurrences (total events)
ev_absolute = dfs['Activity'].value_counts()
# per-case activity counts: dataframe with one row per (case, activity)
case_activity_counts = dfs.groupby(['Case ID', 'Activity']).size().reset_index(name='count')
# case frequency: number of distinct cases containing the activity
ev_case_freq = case_activity_counts.groupby('Activity')['Case ID'].nunique()
# max repetitions: max occurrences of activity within a single case
ev_max_reps = case_activity_counts.groupby('Activity')['count'].max()
# case coverage: percentage of cases containing the activity
ev_case_coverage = (ev_case_freq / num_cases) * 100.0

In [135]:
ev_case_freq

Activity
Call Outbound      377
Email Outbound     380
Handle Case        532
Handle Email       428
Inbound Call      3342
Inbound Email      388
Name: Case ID, dtype: int64

In [136]:
def get_event_metric_series(mode):
  if mode == 'absolute':
    return ev_absolute
  if mode == 'case':
    return ev_case_freq
  if mode == 'max':
    return ev_max_reps
  if mode == 'coverage':
    return ev_case_coverage

In [137]:
dfs_traces = (dfs.sort_values(by=['Case ID', 'start']).groupby(['Case ID']).agg({'Activity': lambda x: tuple(x)}))
# build per-case pair counters
per_case_pairs = {}
for case_id, row in dfs_traces.iterrows():
  trace = row['Activity']
  pairs = list(pairwise(trace))
  per_case_pairs[case_id] = Counter(pairs)

In [138]:
# aggregate flow metrics
flow_abs = Counter()
flow_case_freq = Counter()
flow_max = Counter()
for case_id, cnt in per_case_pairs.items():
  for pair, c in cnt.items():
    flow_abs[pair] += c
    if c > 0:
      flow_case_freq[pair] += 1
    flow_max[pair] = max(flow_max.get(pair, 0), c)

flow_case_coverage = {pair: (flow_case_freq[pair] / num_cases) * 100.0 for pair in flow_abs.keys()}


In [139]:
def get_flow_metric_dict(mode):
  if mode == 'absolute':
    return dict(flow_abs)
  if mode == 'case':
    return dict(flow_case_freq)
  if mode == 'max':
    return dict(flow_max)
  if mode == 'coverage':
    return dict(flow_case_coverage)

In [140]:
# collect start/end events
ev_start_set = set()
ev_end_set = set()
for case_id, row in dfs_traces.iterrows():
  trace = row['Activity']
  if len(trace) == 0:
    continue
  ev_start_set.add(trace[0])
  ev_end_set.add(trace[-1])

ev_start_edges = {}
for ev_start in ev_start_set:
  edge_label = sum(1 for _, row in dfs_traces.iterrows() if row['Activity'] and row['Activity'][0] == ev_start)
  ev_start_edges[ev_start] = edge_label
ev_end_edges = {}
for ev_end in ev_end_set:
  edge_label = sum(1 for _, row in dfs_traces.iterrows() if row['Activity'] and row['Activity'][-1] == ev_end)
  ev_end_edges[ev_end] = edge_label


Probably half of this function could be pruned, but I guess it works.

In [141]:
def render_graphs(mode, filter_by='none', thr_ev=0.0, thr_flow=0.0):
  """Render both simple and weighted DFG images using the chosen mode and optional thresholds.

  mode: one of 'absolute','case','max','coverage'
  filter_by: 'none','events','flows','both'
  thr_ev / thr_flow: numeric thresholds (for coverage use percent 0-100)
  """
  event_metric = get_event_metric_series(mode)
  flow_metric = get_flow_metric_dict(mode)

  # build DFG mapping using chosen flow metric
  dfg = {}
  for (a, b), val in flow_metric.items():
    if a not in dfg:
      dfg[a] = Counter()
    dfg[a][b] = val

  # determine color bounds from event_metric
  event_metric_values = [v for v in (event_metric.reindex(index=dfs['Activity'].unique(), fill_value=0).to_dict().values())]
  if len(event_metric_values) > 0:
    color_min = min(event_metric_values)
    color_max = max(event_metric_values)
  else:
    color_min = 0
    color_max = 1

  filter_events = filter_by in ('events', 'both')
  filter_flows = filter_by in ('flows', 'both')

  # Weighted visualization with nodes and edge labels
  trace_counts = sorted(chain(*[c.values() for c in dfg.values()])) if len(dfg) > 0 else [0, 1]
  trace_min = trace_counts[0]
  trace_max = trace_counts[-1]

  Gw = pgv.AGraph(strict=False, directed=True)
  Gw.graph_attr['rankdir'] = 'LR'
  Gw.node_attr['shape'] = 'Mrecord'

  Gw.add_node("start", shape="circle", label="")
  for ev_start in sorted(ev_start_set):
    if filter_events and event_metric.get(ev_start, 0) < thr_ev:
      continue

    # find how many traces start with this event
    edge_label = ev_start_edges[ev_start]

    Gw.add_edge("start", ev_start, penwidth=3.5, label=edge_label, style="dotted")

  for event, succesors in dfg.items():
    ev_val = event_metric.get(event, 0)
    if filter_events and ev_val < thr_ev:
      continue

    # color scale; avoid division by zero
    if color_max != color_min:
      color = int(float(color_min - ev_val) / float(color_min - color_max) * 100.00)
    else:
      color = 5
    
    my_color = "#ff9933" + str(hex(max(0, min(255, color)))[2:])
    label_val = format_label_value(ev_val, mode)
    Gw.add_node(event, style="rounded,filled", fillcolor=my_color, label=f"{event} ({label_val})")
    
    for succesor, cnt in succesors.items():
      if filter_flows and cnt < thr_flow:
        continue
      if filter_events and event_metric.get(succesor, 0) < thr_ev:
        continue
      
      lbl = format_label_value(cnt, mode)
      pen = 4 * float(cnt) / (trace_max - trace_min + 1) + 0.1
      Gw.add_edge(event, succesor, penwidth=min(pen, 5.0), label=lbl)

  Gw.add_node("end", shape="circle", label="", penwidth='3')
  for ev_end in sorted(ev_end_set):
    if filter_events and event_metric.get(ev_end, 0) < thr_ev:
      continue
    ev_val = event_metric.get(ev_end, 0)

    # find how many traces end with this event
    edge_label = ev_end_edges[ev_end]

    if color_max != color_min:
      color = int(float(color_min - ev_val) / float(color_min - color_max) * 100.00)
    else:
      color = 0
    my_color = "#ff9933" + str(hex(max(0, min(255, color)))[2:])
    label_val = format_label_value(ev_val, mode)
    Gw.add_node(ev_end, style="rounded,filled", fillcolor=my_color, label=f"{ev_end} ({label_val})")
    Gw.add_edge(ev_end, "end", penwidth=3.5, label=f"{edge_label}", style="dotted")

  Gw.draw('simple_heuristic_net_with_events.png', prog='dot')
  display(Image('simple_heuristic_net_with_events.png'))

  # summary
  total_edges = Gw.number_of_edges()
  sum_edge_labels = 0
  for e in Gw.edges():
    try:
      lab = e.attr['label']
      if lab is None:
        continue
      if isinstance(lab, str) and lab.endswith('%'):
        sum_edge_labels += float(lab.rstrip('%'))
      else:
        sum_edge_labels += float(lab)
    except Exception:
      continue
  print(f"Mode: {mode}. Filter: {filter_by}. Thr-events: {thr_ev}. Thr-flows: {thr_flow}. Number of edges: {total_edges}, Label sum: {sum_edge_labels}")


In [142]:
mode = "absolute"  # Default mode
filter_by = 'none'  # Default filter
threshold_events = 0.0  # Default threshold for events
threshold_flows = 0.0  # Default threshold for flows

mode_dropdown = widgets.Dropdown(options=['absolute', 'case', 'max', 'coverage'], value=mode, description='Mode:')
filter_dropdown = widgets.Dropdown(options=['none', 'events', 'flows', 'both'], value=filter_by, description='Filter:')

event_metric = get_event_metric_series(mode)

try:
  max_ev = int(event_metric.max())
except Exception:
  max_ev = 100 if mode == 'coverage' else 1
flow_metric = get_flow_metric_dict(mode)
try:
  max_flow = int(max(flow_metric.values())) if len(flow_metric) > 0 else 1
except Exception:
  max_flow = 1
# coverage mode expects percent thresholds
if mode == 'coverage':
  max_ev = 100
  max_flow = 100

In [143]:
thr_ev_box = widgets.IntSlider(value=min(int(threshold_events), max_ev), min=0, max=max_ev if max_ev>0 else 1, step=1, description='Thr events:')
thr_flow_box = widgets.IntSlider(value=min(int(threshold_flows), max_flow), min=0, max=max_flow if max_flow>0 else 1, step=1, description='Thr flows:')

In [144]:
def _on_mode_change(change):
  if change.get('name') != 'value':
    return
  new_mode = change.get('new')
  ev_m = get_event_metric_series(new_mode)
  try:
    new_max_ev = int(ev_m.max())
  except Exception:
    new_max_ev = 100 if new_mode == 'coverage' else 1
  flow_m = get_flow_metric_dict(new_mode)
  try:
    new_max_flow = int(max(flow_m.values())) if len(flow_m) > 0 else 1
  except Exception:
    new_max_flow = 1
  if new_mode == 'coverage':
    new_max_ev = 100
    new_max_flow = 100
  # set slider maxima and clamp values if needed
  thr_ev_box.max = new_max_ev if new_max_ev > 0 else 1
  thr_flow_box.max = new_max_flow if new_max_flow > 0 else 1
  if thr_ev_box.value > thr_ev_box.max:
    thr_ev_box.value = thr_ev_box.max
  if thr_flow_box.value > thr_flow_box.max:
    thr_flow_box.value = thr_flow_box.max

In [145]:
mode_dropdown.observe(_on_mode_change, names='value')
generate_btn = widgets.Button(description='Generate graph')
out = widgets.Output()

In [146]:
def _on_generate_clicked(b):
  # clear previous output (graphs) but keep widgets visible
  out.clear_output(wait=True)
  mode_w = mode_dropdown.value
  filter_w = filter_dropdown.value
  thr_ev_w = float(thr_ev_box.value)
  thr_flow_w = float(thr_flow_box.value)
  # remove old graph files if present to avoid stale images
  for fn in ('simple_heuristic_net.png', 'simple_heuristic_net_with_events.png'):
    try:
      if os.path.exists(fn):
        os.remove(fn)
    except Exception:
      pass
  with out:
    render_graphs(mode_w, filter_w, thr_ev_w, thr_flow_w)

generate_btn.on_click(_on_generate_clicked)

In [147]:
ui = widgets.HBox([mode_dropdown, filter_dropdown, thr_ev_box, thr_flow_box, generate_btn])
display(ui)
display(out)

Output()

In [148]:
def prepare_dict_of_neighbours(flow_metric, ev_start_edges, ev_end_edges):
    neighbours_dict = {}
    for edge, value in flow_metric.items():
        a, b = edge
        if a not in neighbours_dict:
            neighbours_dict[a] = {}
        neighbours_dict[a][b] = value

    #TODO @lursz plz fix
    start_node = "START"
    end_node = "END"

    neighbours_dict[start_node] = {}
    for event in ev_start_edges:
        neighbours_dict[start_node][event] = ev_start_edges[event]
    for event in ev_end_edges:
        if event not in neighbours_dict:
            neighbours_dict[event] = {}
        neighbours_dict[event][end_node] = ev_end_edges[event]

    for node, events in neighbours_dict.items():
        neighbours_dict[node] = OrderedDict(sorted(events.items(), key=lambda x: x[1], reverse=True))

    reverse_neighbours_dict = {}
    for a in neighbours_dict:
        for b in neighbours_dict[a]:
            if b not in reverse_neighbours_dict:
                reverse_neighbours_dict[b] = {}
            reverse_neighbours_dict[b][a] = neighbours_dict[a][b]

    for node, events in reverse_neighbours_dict.items():
        reverse_neighbours_dict[node] = OrderedDict(sorted(events.items(), key=lambda x: x[1], reverse=True))

    return neighbours_dict, reverse_neighbours_dict

In [176]:
from collections import deque

def bfs_shortest_distance(neighbours_dict, start_node):
    visited = {start_node: 0}
    queue = deque([start_node])

    while queue:
        current_node = queue.popleft()

        for neighbour in neighbours_dict.get(current_node, {}):
            if neighbour not in visited:
                visited[neighbour] = visited[current_node] + 1
                queue.append(neighbour)

    return visited

def bfs_shortest_path(neighbours_dict, start_nodes, end_nodes):
    if type(start_nodes) is not list:
        start_nodes = [start_nodes]
    visited = {node: node for node in start_nodes}
    queue = deque(start_nodes)

    final_path = []
    while queue:
        current_node = queue.popleft()

        for neighbour in neighbours_dict.get(current_node, {}):
            if neighbour not in visited:
                visited[neighbour] = current_node
                if neighbour in end_nodes:
                    final_path.append(neighbour)
                    final_path.append(current_node)
                    queue.clear()
                    break
                queue.append(neighbour)

    while True:
        if final_path[-1] in start_nodes:
            break
        final_path.append(visited[final_path[-1]])
    return final_path
    

def find_most_valuable_path(key_nodes, neighbours_dict, reverse_neighbours_dict, start_node, bfs_reverse_distances):
    visited = {start_node: (0, start_node)}
    queue = deque([start_node])

    while queue:
        current_node = queue.popleft()
        current_value, _ = visited[current_node]

        node_value = key_nodes.get(current_node, 0)
        for neighbour in reverse_neighbours_dict.get(current_node, {}):
            if neighbour not in key_nodes:
                continue
            if bfs_reverse_distances[neighbour] <= bfs_reverse_distances[current_node]:
                continue
            neighbour_value, _ = visited.get(neighbour, (0, None))
            if neighbour_value + node_value > current_value:
                visited[current_node] = (neighbour_value + node_value, neighbour)
        for neighbour in neighbours_dict.get(current_node, {}):
            if neighbour not in key_nodes:
                continue
            if neighbour not in visited:
                visited[neighbour] = (0, current_node)
                queue.append(neighbour)

    final_path = []
    while True:
        final_path.append(current_node)
        _, parent = visited[current_node]
        if parent == current_node:
            break
        current_node = parent
    final_path.reverse()

    return final_path


In [210]:
def find_key_path(event_metric, flow_metric, ev_start_edges, ev_end_edges):
    neighbours_dict, reverse_neighbours_dict = prepare_dict_of_neighbours(flow_metric, ev_start_edges, ev_end_edges)
    bfs_distances = bfs_shortest_distance(neighbours_dict, "START")
    bfs_reverse_distances = bfs_shortest_distance(reverse_neighbours_dict, "END")

    distance_to_end = bfs_distances["END"]
    key_nodes = {node: int(event_metric.get(node, 0)) for node in bfs_distances if node in bfs_reverse_distances if bfs_distances[node] + bfs_reverse_distances[node] == distance_to_end}

    key_path = find_most_valuable_path(key_nodes, neighbours_dict, reverse_neighbours_dict, "START", bfs_reverse_distances)

    key_edges = [edge for edge in pairwise(key_path)]

    return key_path, key_edges

In [260]:
def filter(node_percentage, node_threshold, all_edges, edge_percentage, edge_threshold):
    # filter nodes
    nodes = []
    for node, value in node_percentage.items():
        if value > node_threshold:
            break
        nodes.append(node)
    
    # find edges that MUST be included
    crucial_edges = []
    for edge, value in edge_percentage.items():
        if value > node_threshold:
            break
        crucial_edges.append(edge)

    # find remaining edges that connect the nodes, that are currently included
    remaining_edges = set(all_edges) - set(edge_percentage)
    edges_to_consider = []
    for edge in remaining_edges:
        a, b = edge
        if a in nodes and b in nodes:
            edges_to_consider.append(edge)

    # sort by value descending
    remaining_edges = sorted(edges_to_consider, key=lambda x: edge_percentage.get(x, 0), reverse=True)

    # add edges until threshold is reached
    edge_count = len(crucial_edges)
    denominator = len(set(crucial_edges).union(set(remaining_edges)))
    for i, edge in enumerate(remaining_edges):
        if (1 + i + edge_count) / denominator * 100.0 > edge_threshold:
            break
        crucial_edges.append(edge)

    return nodes, crucial_edges

In [261]:
neighbours_dict, reverse_neighbours_dict = prepare_dict_of_neighbours(flow_metric, ev_start_edges, ev_end_edges)
key_path, key_edges = find_key_path(event_metric, flow_metric, ev_start_edges, ev_end_edges)

events_filtered = list((event, value) for event, value in event_metric.items() if event not in key_path)
events_filtered = OrderedDict(sorted(events_filtered, key=lambda item: item[1], reverse=True))

nodes_percentage = {event: 0 for event in key_path}
node_count = len(event_metric)
# when setting node filtering percentage, those edges must also be added
edges_percentage = {edge: 0 for edge in key_edges}

for event in events_filtered:
    if event in key_path:
        continue
    shortest_paths = bfs_shortest_path(neighbours_dict, event, set(key_path))
    path_edges = [edge for edge in pairwise(reversed(shortest_paths))]
    shortest_paths_rev = bfs_shortest_path(neighbours_dict, key_path, event)
    path_edges_rev = [edge for edge in pairwise(reversed(shortest_paths_rev))]
    print(path_edges, path_edges_rev)

    nodes_to_add = set(shortest_paths + shortest_paths_rev) - set(key_path)
    percentage = round((len(nodes_to_add) + len(key_path) - 2) / node_count * 100.0, 2)

    key_path = key_path + list(nodes_to_add)
    nodes_percentage.update({node: percentage for node in nodes_to_add})

    edges_to_add = set(key_edges + list(path_edges) + list(path_edges_rev)) - set(edges_percentage)
    key_edges = key_edges + list(edges_to_add)
    edges_percentage.update({edge: percentage for edge in edges_to_add})

[('Inbound Call', 'END')] [('START', 'Inbound Call')]
[('Handle Case', 'END')] [('START', 'Handle Case')]
[('Call Outbound', 'END')] [('START', 'Call Outbound')]
[('Handle Email', 'END')] [('START', 'Handle Email')]
[('Email Outbound', 'Handle Email')] [('START', 'Email Outbound')]


In [ ]:
x, y = filter(nodes_percentage, 100.0, flow_metric, edges_percentage, 40.0)
x, y

(['START',
  'Inbound Email',
  'END',
  'Inbound Call',
  'Handle Case',
  'Call Outbound',
  'Handle Email',
  'Email Outbound'],
 [('START', 'Inbound Email'),
  ('Inbound Email', 'END'),
  ('Inbound Call', 'END'),
  ('START', 'Inbound Call'),
  ('Handle Case', 'END'),
  ('START', 'Handle Case'),
  ('Call Outbound', 'END'),
  ('START', 'Call Outbound'),
  ('Handle Email', 'END'),
  ('START', 'Handle Email'),
  ('START', 'Email Outbound'),
  ('Email Outbound', 'Handle Email'),
  ('Handle Email', 'Call Outbound'),
  ('Inbound Call', 'Inbound Email'),
  ('Call Outbound', 'Inbound Email'),
  ('Inbound Email', 'Inbound Call'),
  ('Handle Case', 'Inbound Call'),
  ('Email Outbound', 'Inbound Email')])